In [4]:
!pip install gradio scipy imbalanced-learn mlxtend


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.2/298.2 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 8.3 MB/s eta 0:

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import pandas as pd

In [8]:
path = "/content/drive/MyDrive/Capstone Project /"

test = pd.read_csv(path + 'Test.csv')
sample_submission = pd.read_csv(path + 'SampleSubmission.csv')
train = pd.read_csv(path + 'Train.csv')
variable_definitions = pd.read_csv(path + 'VariableDefinitions.csv')

In [9]:
# -*- coding: utf-8 -*-
"""Elly 2.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1WFSE8UyWKXZ9AZv81U-QaCWLTxe8I5kn
"""

# Commented out IPython magic to ensure Python compatibility.
# Data Manipuulation
import pandas as pd
import numpy as np

# Vizualisation (Matplotlib, Plotly, Seaborn, etc. )
# %matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display

# Feature Processing (Scikit-learn processing, etc. )
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix , classification_report, f1_score, accuracy_score,precision_score, recall_score, fbeta_score, make_scorer, roc_auc_score
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.model_selection import GridSearchCV, cross_validate

from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import roc_auc_score,roc_curve

#from skopt import BayesSearchCV
from sklearn.utils import class_weight

import scipy.stats as stats

# models
from sklearn import svm
#from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier


from imblearn.over_sampling import  RandomOverSampler
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.model_selection import KFold, StratifiedKFold, RepeatedStratifiedKFold


# model interpretation
#import shap
import joblib
import os
import warnings
warnings.filterwarnings("ignore")

# Load the Train dataset
#train = pd.read_csv('C:\Users\user\Desktop\ChurnAnalysis_CapstonePred\Train.csv')
#test = pd.read_csv('C:\Users\user\Desktop\ChurnAnalysis_CapstonePred\Test.csv')


# create a functioon to replace the missing values with mean for each respective column in the train data
columns_to_fill = ['MONTANT', 'FREQUENCE_RECH', 'REVENUE', 'ARPU_SEGMENT',
                   'FREQUENCE', 'DATA_VOLUME', 'ON_NET', 'ORANGE', 'TIGO',
                   'FREQ_TOP_PACK']

for column in columns_to_fill:
    train[column].fillna(train[column].mean(), inplace=True)

# create a functioon to replace the missing values with mean for each respective column in the test data
columns_to_fill = ['MONTANT', 'FREQUENCE_RECH', 'REVENUE', 'ARPU_SEGMENT',
                   'FREQUENCE', 'DATA_VOLUME', 'ON_NET', 'ORANGE', 'TIGO',
                   'FREQ_TOP_PACK']

for column in columns_to_fill:
    test[column].fillna(test[column].mean(), inplace=True)

# Create a DataFrame for the train data
train = pd.DataFrame(train)

# Fill missing values based on the frequency distribution in the train data
def fill_missing_with_distribution(column):
    freq_distribution = column.value_counts(normalize=True)
    missing_count = column.isnull().sum()
    missing_indices = column[column.isnull()].index
    imputed_values = np.random.choice(freq_distribution.index, size=missing_count, p=freq_distribution.values)
    column[missing_indices] = imputed_values

# Perform missing value imputation for each categorical column in the train data
categorical_columns = ['REGION', 'TOP_PACK']

for col in categorical_columns:
    fill_missing_with_distribution(train[col])

# Create a DataFrame for the test data
test = pd.DataFrame(test)

# Fill missing values based on the frequency distribution in the test data
def fill_missing_with_distribution(column):
    freq_distribution = column.value_counts(normalize=True)
    missing_count = column.isnull().sum()
    missing_indices = column[column.isnull()].index
    imputed_values = np.random.choice(freq_distribution.index, size=missing_count, p=freq_distribution.values)
    column[missing_indices] = imputed_values

# Perform missing value imputation for each categorical column
categorical_columns_test = ['REGION', 'TOP_PACK']

for col in categorical_columns_test:
    fill_missing_with_distribution(test[col])



train['ZONE1'].fillna((train['ZONE1'].mean()), inplace=True)
train['ZONE2'].fillna((train['ZONE2'].mean()), inplace=True)

# train.drop(columns=['REGION', 'MRG', 'TOP_PACK'], inplace=True) #drop these columns

# **Machine Learning**

from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import accuracy_score,confusion_matrix,recall_score,precision_recall_curve, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

"""## **Feature Engineering**

### **Feature Selection**
"""

drop =['user_id', 'CHURN', 'REGION', 'MRG', 'TOP_PACK']
y=train['CHURN']
x=train.drop(columns=drop, axis=1)
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size = 0.2)

print(X_train.shape)
print(y_train.shape)

"""### **Splitting Dataset**"""

X_train,X_val,y_train,y_val = train_test_split(X_train,y_train,test_size = 0.3, random_state=1)

xt= X_train
yt= y_train

"""### **Standize Numeric Columns**"""

num_cols = ['MONTANT', 'FREQUENCE_RECH', 'REVENUE', 'ARPU_SEGMENT', 'FREQUENCE',
       'DATA_VOLUME', 'ON_NET', 'ORANGE', 'TIGO',
       'REGULARITY', 'FREQ_TOP_PACK']
      #  'ZONE1', 'ZONE2',

scaler = StandardScaler()
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_train.head()
X_test[num_cols] = scaler.transform(X_test[num_cols])
X_val[num_cols] = scaler.transform(X_val[num_cols])

"""### **Encode the Tenure Column**"""

encoder = LabelEncoder()
X_train["TENURE"] = encoder.fit_transform(X_train["TENURE"])
X_test["TENURE"] = encoder.transform(X_test["TENURE"])
X_val['TENURE'] = encoder.transform(X_val["TENURE"])

# Due to high number of missing data in Zone 1 and Zone 2 their column would be dropped

X_train.drop(columns=['ZONE1', 'ZONE2'], inplace=True)
X_test.drop(columns=['ZONE1', 'ZONE2'], inplace=True)
X_val.drop(columns=['ZONE1', 'ZONE2'], inplace=True)

"""### **Balancing Dataset**"""

x_train_randOverSample = X_train.copy()
y_train_randOverSample = y_train.copy()


randOverSample = RandomOverSampler(random_state=100,)
X_train_randOverSample, y_train_randOverSample = randOverSample.fit_resample(x_train_randOverSample, y_train_randOverSample.ravel())


"""## Creating Models"""

last_models_metrics = []

classifiers = [('LR', LogisticRegression()),
                   ("DT", DecisionTreeClassifier()),
                   ("RF", RandomForestClassifier()),
                   ('GBM', GradientBoostingClassifier()),
                   ('XGBoost', XGBClassifier(use_label_encoder=False, eval_metric='logloss')),

                   ]
for name, classifier in classifiers:
        cv_results = cross_validate(classifier, X_train_randOverSample, y_train_randOverSample, cv=3, scoring=["accuracy", "f1", "roc_auc", "precision", "recall"])
        accuracy = round(cv_results['test_accuracy'].mean(), 4)
        auc = round(cv_results['test_roc_auc'].mean(), 4)
        recall = round(cv_results['test_recall'].mean(), 4)
        precision = round(cv_results['test_precision'].mean(), 4)
        f1 = round(cv_results['test_f1'].mean(), 4)

        last_models_metrics.append({
        "Model": name,
        "Accuracy": accuracy,
        "AUC": auc,
        "Recall": recall,
        "Precision": precision,
        "F1": f1
          })

metrics = {
    "Model": ["LR", "DT", "RF", "LightGBM", "XGB", ],
    "Accuracy": [last_models_metrics[0]['Accuracy'], last_models_metrics[1]['Accuracy'],
                 last_models_metrics[2]['Accuracy'], last_models_metrics[3]['Accuracy'],
                 last_models_metrics[4]['Accuracy'], ],
    "AUC": [last_models_metrics[0]['AUC'], last_models_metrics[1]['AUC'],
            last_models_metrics[2]['AUC'], last_models_metrics[3]['AUC'],
            last_models_metrics[4]['AUC'], ],
    "Recall": [last_models_metrics[0]['Recall'], last_models_metrics[1]['Recall'],
               last_models_metrics[2]['Recall'], last_models_metrics[3]['Recall'],
               last_models_metrics[4]['Recall'],],
    "Precision": [last_models_metrics[0]['Precision'], last_models_metrics[1]['Precision'],
                  last_models_metrics[2]['Precision'], last_models_metrics[3]['Precision'],
                  last_models_metrics[4]['Precision'],],
    "F1": [last_models_metrics[0]['F1'], last_models_metrics[1]['F1'],
           last_models_metrics[2]['F1'], last_models_metrics[3]['F1'],
           last_models_metrics[4]['F1'], ]
}

"""### **Feature Importance**"""

models = [RandomForestClassifier(),
              XGBClassifier(),
            DecisionTreeClassifier()
]


"""### **Hyperparameter Optimization**"""

lr_params = {"C": [0.001, 0.01, 0.1, 1, 10 ]}
knn_params = {"n_neighbors": [3, 5]}
dt_params = {"max_depth": [3, 5, 7]}
rf_params = {"n_estimators": [10]}
xgb_params = {"learning_rate": [0.01, 0.1], "max_depth": [3]}

classifiers = [
    ("LR", LogisticRegression(), lr_params),

    ("DT", DecisionTreeClassifier(), dt_params),
    ("RF", RandomForestClassifier(), rf_params),
     ("XGB", XGBClassifier(), xgb_params)
    ]

def hyperparameters_optimization(X_train_randOverSample, y_train_randOverSample, cv=5, scoring="f1"):
    print("Hyperparameter Optimimization")
    best_models = {}
    for name, classifier, params in classifiers:

        gs_best = GridSearchCV(classifier, params, cv=cv, n_jobs=-1, verbose=False).fit(X_train_randOverSample,y_train_randOverSample)
        final_model = classifier.set_params(**gs_best.best_params_)

        cv_results = cross_validate(final_model, X_train_randOverSample, y_train_randOverSample, cv=cv, scoring=scoring)
        print(f"{scoring} (After) : {round(cv_results['test_score'].mean(), 4)}")
        print(f"{name} best params: {gs_best.best_params_}", end="\n\n")
        best_models[name] = final_model
    return best_models

best_models = hyperparameters_optimization(X_train_randOverSample, y_train_randOverSample)

"""### **Final Model**

Based on the above result ,Random Forest would be selected as the final model
"""

best_models['RF']

RF_model = best_models['RF'].fit(X_train_randOverSample, y_train_randOverSample)

# Prediction with Trained Model

y_pred = RF_model.predict(X_test)
y_prob = RF_model.predict_proba(X_test)[:, 1]
#print(classification_report(y_test, y_pred))

RF_roc_auc = roc_auc_score(y_test, y_prob)
#fpr, tpr, thresholds = roc_curve(y_test, y_prob)
#plt.figure()

# plt.plot([0,1],[0,1],'r--')
# plt.plot(fpr, tpr, marker='.', label='RF')
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title("Random Forest ROC")
# plt.legend()
# plt.show()

"""### **Saving Model**"""

joblib.dump(best_models['RF'], 'best_model_filename.pkl')



(861619, 14)
(861619,)
Hyperparameter Optimimization
########## LR #########
f1 (After) : 0.8324
LR best params: {'C': 1}

########## DT #########
f1 (After) : 0.8338
DT best params: {'max_depth': 7}

########## RF #########
f1 (After) : 0.9051
RF best params: {'n_estimators': 10}

########## XGB #########
f1 (After) : 0.8333
XGB best params: {'learning_rate': 0.1, 'max_depth': 3}



['best_model_filename.pkl']

In [16]:
X_test.head(2)

,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,REGULARITY,FREQ_TOP_PACK
713703,7,0.000678,0.001120,0.000570,0.000569,0.000607,0.00188,0.001649,0.000787,0.000804,-1.122767,0.000370
622779,7,0.922294,1.450304,0.967845,0.967844,1.591502,0.04830,-0.394595,-0.149192,-0.277542,1.480458,0.507414


# Check if the model works

In [30]:
MODEL = best_models['RF']

In [31]:
# Predictions using the trained model
pred = MODEL.predict(X_test)

pred[0]

1

# Gradio Build

In [35]:
import gradio as gr
import pandas as pd

# Define a function to classify the output
def classifier_1(result):
    if result == 0:
        return "Customer will churn"
    else:
        return "Customer will not churn"

# Map numerical values to labels
tenure_labels = {
    0: "3-6 months",
    1: "6-9 months",
    2: "9-12 months",
    3: "12-15 months",
    4: "15-18 months",
    5: "18-21 months",
    6: "21-24 months",
    7: "> 24 months"
}

# Reverse the mapping for predictions
tenure_values = {v: k for k, v in tenure_labels.items()}

def predict(tenure, montant, freq_rech, revenue, arpu, freq, data_vol, on_net, orange, tigo, zone1, zone2, regularity, freq_top_pack):
    inputs = [tenure, montant, freq_rech, revenue, arpu, freq, data_vol, on_net, orange, tigo, zone1, zone2, regularity, freq_top_pack]

    input_df = pd.DataFrame([inputs], columns=['tenure', 'montant', 'freq_rech', 'revenue', 'arpu',
                                               'freq', 'data_vol', 'on_net', 'orange', 'tigo',
                                               'zone1', 'zone2', 'regularity', 'freq_top_pack'])
    # Convert dropdown label to numerical value
    tenure_value = tenure_values[tenure]

    pred = MODEL.predict(input_df)

    output = classifier_1(pred[0])

    # if output == "Customer will churn":
    #     return [(0, output)]
    # else:
    #     return [(1, output)]

    if output == "Customer will churn":
        return {"prediction": output}
    else:
        return {"prediction": output}

# Define the outputs
output = gr.outputs.HighlightedText(color_map={
    "Customer will churn": "green",
    "Customer will not churn": "red"
})

# Create a dropdown menu with labels
tenure_dropdown = gr.inputs.Dropdown(list(tenure_labels.values()), label="TENURE")

iface = gr.Interface(
    fn=predict,
    inputs=[
        tenure_dropdown,  # Dropdown instead of slider
        #gr.inputs.Slider(minimum=1, maximum=7, label="TENURE"),
        gr.inputs.Slider(minimum=20, maximum=470000, label="MONTANT"),
        gr.inputs.Slider(minimum=1, maximum=131, label="FREQUENCE_RECH"),
        gr.inputs.Slider(minimum=1, maximum=530000, label="REVENUE"),
        gr.inputs.Slider(minimum=0, maximum=2453, label="ARPU_SEGMENT"),
        gr.inputs.Slider(minimum=1, maximum=91, label="FREQUENCE"),
        gr.inputs.Slider(minimum=1, maximum=1702309, label="DATA_VOLUME"),
        gr.inputs.Slider(minimum=0, maximum=51000, label="ON_NET"),
        gr.inputs.Slider(minimum=0, maximum=12040, label="ORANGE"),
        gr.inputs.Slider(minimum=0, maximum=4174, label="TIGO"),
        #gr.inputs.Slider(minimum=0, maximum=2510, label="ZONE1"),
        #gr.inputs.Slider(minimum=0, maximum=3697, label="ZONE2"),
        gr.inputs.Slider(minimum=0, maximum=62, label="REGULARITY"),
        gr.inputs.Slider(minimum=0, maximum=624, label="FREQ_TOP_PACK")
    ],
    outputs=output,
    title="Team Paris Customer Churn Prediction App",
    description="Let's Get Started With Some Predictions!"
)

iface.launch()


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

# Into pipeline

In [13]:
# Import the necessary libraries
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import FunctionTransformer
import joblib

# Define a preprocessing pipeline
preprocessing_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),  # Fill missing values with mean
    ('scaler', StandardScaler()),  # Standardize numeric columns
    ('encoder', LabelEncoder()),  # Encode the TENURE column
    ('column_dropper', FunctionTransformer(lambda data: data.drop(columns=['ZONE1', 'ZONE2'], axis=1), validate=False)),  # Drop specified columns
    ('oversampler', RandomOverSampler(random_state=100)),  # Balance the dataset
])

# Define the final model (Random Forest)
final_model = RandomForestClassifier()

# Create a full pipeline by combining preprocessing and the final model
full_pipeline = Pipeline([
    ('preprocessing', preprocessing_pipeline),
    ('model', final_model),
])

# Fit the full pipeline to your training data
X_train_randOverSample, y_train_randOverSample = preprocessing_pipeline.fit(X_train, y_train) #_resample
full_pipeline.fit(X_train_randOverSample, y_train_randOverSample)

# Save the entire pipeline, including preprocessing and the final model
joblib.dump(full_pipeline, 'full_pipeline_with_columns_dropped.pkl')


TypeError: ignored